**1. 安装PaddleX**

In [ ]:
!pip install paddlex==2.1.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [ ]:
! wget https://bj.bcebos.com/paddlex/datasets/vegetables_cls.tar.gz
! tar xzf vegetables_cls.tar.gz



--2022-02-23 20:44:13--  https://bj.bcebos.com/paddlex/datasets/vegetables_cls.tar.gz
Resolving bj.bcebos.com (bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to bj.bcebos.com (bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26655831 (25M) [application/octet-stream]
Saving to: ‘vegetables_cls.tar.gz’

vegetables_cls.tar. 100%[===================>]  25.42M  19.6MB/s    in 1.3s    

2022-02-23 20:44:14 (19.6 MB/s) - ‘vegetables_cls.tar.gz’ saved [26655831/26655831]



**![](https://ai-studio-static-online.cdn.bcebos.com/92de3951cece4122b1a10f386733ae0776c62b5cecfc46c3ac911d6548b8510e)2. 准备蔬菜分类数据集**


In [ ]:
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx

In [ ]:
from paddlex import transforms
train_transforms = transforms.Compose([
    transforms.RandomCrop(crop_size=224),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.ResizeByShort(short_size=256),
    transforms.CenterCrop(crop_size=224),
    transforms.Normalize()
])

In [ ]:
import paddlex as pdx
train_dataset = pdx.datasets.ImageNet(
    data_dir='vegetables_cls',
    file_list='vegetables_cls/train_list.txt',
    label_list='vegetables_cls/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.ImageNet(
    data_dir='vegetables_cls',
    file_list='vegetables_cls/val_list.txt',
    label_list='vegetables_cls/labels.txt',
    transforms=eval_transforms)

2022-02-23 20:50:50 [INFO]	Starting to read file list from dataset...
2022-02-23 20:50:50 [INFO]	840 samples in file vegetables_cls/train_list.txt
2022-02-23 20:50:50 [INFO]	Starting to read file list from dataset...
2022-02-23 20:50:50 [INFO]	240 samples in file vegetables_cls/val_list.txt


**模型开始训练

In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.cls.MobileNetV2(num_classes=num_classes)
model.train(num_epochs=10,
            train_dataset=train_dataset,
            train_batch_size=32,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[4, 6, 8],
            save_interval_epochs=1,
            learning_rate=0.025,
            save_dir='output/mobilenetv2',
            use_vdl=True)

2022-02-23 20:51:21 [INFO]	Downloading MobileNetV2_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/MobileNetV2_pretrained.pdparams


100%|██████████| 20851/20851 [00:01<00:00, 14916.63KB/s]


2022-02-23 20:51:23 [INFO]	Loading pretrained model from output/mobilenetv2/pretrain/MobileNetV2_pretrained.pdparams
2022-02-23 20:51:23 [WARNING]	[SKIP] Shape of pretrained params out.weight doesn't match.(Pretrained: [1280, 1000], Actual: [1280, 6])
2022-02-23 20:51:23 [WARNING]	[SKIP] Shape of pretrained params out.bias doesn't match.(Pretrained: [1000], Actual: [6])
2022-02-23 20:51:23 [INFO]	There are 265/267 variables loaded into MobileNetV2.
2022-02-23 20:52:37 [INFO]	[TRAIN] Epoch=1/10, Step=10/26, loss=0.254791, acc1=0.937500, acc5=1.000000, lr=0.025000, time_each_step=7.42s, eta=0:31:53


**模型预测

In [1]:
import paddlex as pdx
model = pdx.load_model('output/mobilenetv2/best_model')
image_name = 'vegetables_cls/bocai/100.jpg'
result = model.predict(image_name)
print("Predict Result:", result)

[02-23 21:11:22 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: Depreca

2022-02-23 21:11:23 [ERROR]	model_dir 'output/mobilenetv2/best_model' does not exists!


SystemExit: -1